In [15]:
import pandas as pd
from scipy.stats import spearmanr

In [17]:
#contingency table
df_play = pd.read_csv('fimo.tsv',delimiter='\t')
df_play.fillna('NA',inplace=True)
cont_table_fimo = pd.crosstab(df_play['sequence_name'],df_play['motif_alt_id'])#, margins=True)
print(cont_table_fimo)
print(df_play)
cont_table_fimo.to_csv('cont_table_sea_raw.tsv',sep='\t')
cont_table_fimo.to_excel('cont_table_sea_raw.xlsx',engine='openpyxl')

C:\Users\krish\AppData\Local\Temp\ipykernel_18220\950525610.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'NA' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_play.fillna('NA',inplace=True)


motif_ALT_ID       ABF2  ABI3  ABI5  ABR1  AHL12  AHL25  ANL2  AP3  ARF14  \
seq_ID                                                                      
Glyma.01G043300.1     0     0     0     0      2      2     0    0      2   
Glyma.01G053800.1     2     1     2     0      0      0     1    0      4   
Glyma.01G056800.1     1     2     2     0      1      1     0    0      0   
Glyma.01G128100.1     0     0     0     1      0      0     0    0      0   
Glyma.01G189100.1     0     2     0     2      1      1     1    0      2   
...                 ...   ...   ...   ...    ...    ...   ...  ...    ...   
Glyma.19G254800.1     0     0     0     0      0      0     0    0      4   
Glyma.20G028000.1     0     1     0     0      4      4     0    0      2   
Glyma.20G030500.1     0     2     0     2      1      1     0    0      1   
Glyma.20G163200.1     1     1     1     0      0      0     1    0      5   
NA                    0     0     0     0      0      0     0    0      0   

In [ ]:
import numpy as np
from scipy.stats import spearmanr

# Load your contingency table, skipping the first column (gene names)
# Replace 'contingency_table.csv' with your actual file
contingency_table = pd.read_excel('cont_table_fimo.xlsx', index_col=0)

# Drop the first column if it's the gene names column
tf_table = contingency_table.iloc[:, 1:]  # Exclude the first column

# Define the columns of interest
columns_of_interest = ['CAI', 'rENC']

# Prepare to store results
significant_correlations = []

# Iterate through each column of interest
for col in columns_of_interest:
    for tf in tf_table.columns:
        if tf != col:
            # Calculate Spearman's rank correlation and p-value
            corr, p_value = spearmanr(tf_table[col], tf_table[tf])
            
            # Filter based on statistical significance (e.g., p < 0.05)
            if p_value < 0.05:
                significant_correlations.append((col, tf, corr, p_value))

# Convert the results to a DataFrame for better readability
significant_df = pd.DataFrame(significant_correlations, columns=['Column_of_Interest', 'TF', 'Correlation', 'P-Value'])

# Sort by the strength of the correlation
significant_df = significant_df.sort_values(by='Correlation', ascending=False)

# Save the significant correlations to a file
significant_df.to_csv('significant_cai_nc_correlations_spearman.csv', index=False)

# Display the top few results
print(significant_df.head())


In [ ]:
import pandas as pd
from scipy.stats import spearmanr

# Load your contingency table, skipping the first column (gene names)
# Replace 'contingency_table.xlsx' with your actual file
contingency_table = pd.read_excel('cont_table_fimo.xlsx', index_col=0)

# Drop the first column if it's the gene names column
tf_table = contingency_table.iloc[:, 1:]  # Exclude the first column

# Define the columns of interest
columns_of_interest = ['rENC', 'CAI']

# Prepare dictionaries to store significant correlations
significant_enc = {}
significant_cai = {}

# Function to calculate significant correlations
def calculate_significant_correlations(column):
    significant = {}
    for tf in tf_table.columns:
        if tf != column:
            corr, p_value = spearmanr(tf_table[column], tf_table[tf])
            if p_value < 0.05:
                significant[tf] = (corr, p_value)
    return significant

# Calculate significant correlations with ENC
significant_enc = calculate_significant_correlations('rENC')

# Calculate significant correlations with CAI
significant_cai = calculate_significant_correlations('CAI')

# Find common TFs that are significant with both ENC and CAI
common_tfs = set(significant_enc.keys()).intersection(set(significant_cai.keys()))

# Prepare the results
common_tfs_results = []

for tf in common_tfs:
    enc_corr, enc_p_value = significant_enc[tf]
    cai_corr, cai_p_value = significant_cai[tf]
    common_tfs_results.append((tf, enc_corr, enc_p_value, cai_corr, cai_p_value))

# Convert the results to a DataFrame for better readability
common_tfs_df = pd.DataFrame(common_tfs_results, columns=['TF', 'ENC_Correlation', 'ENC_P-Value', 'CAI_Correlation', 'CAI_P-Value'])

# Sort by the strength of the correlation with ENC
common_tfs_df = common_tfs_df.sort_values(by='ENC_Correlation', ascending=False)

# Save the results to a file
common_tfs_df.to_csv('common_tfs_with_significant_correlations.csv', index=False)

# Display the results
print(common_tfs_df)
